# 2018-04-26 - Améliorer le réseau
On reprend le notebook de la veille et on améliore les performances, avec notamment une séparation correcte des batchs de training et de test. Le réseau convolutionné ne convergeait pas, donc on le rétrécit en un réseau de 3 couches linéaires.

On importe les données :
                   

In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets

data_transform = transforms.Compose(
    [transforms.Grayscale(),
     transforms.Resize((32,32)),
    transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5,0.5), (0.5,0.5,0.5))])

train_set = datasets.ImageFolder(root='clouds_easy',
                                transform=data_transform)
train_loader = torch.utils.data.DataLoader(train_set,
                                             batch_size=4, shuffle=True,
                                             num_workers=1)

test_set = datasets.ImageFolder(root='clouds_easy_test',
                                transform=data_transform)
test_loader = torch.utils.data.DataLoader(test_set,
                                             batch_size=4,shuffle=False,
                                             num_workers=1)
#les 4 thetas qu'on essaie d'apprendre
cloud_classes = ('0', 'pi/4', 'pi/2', '3pi/4')

On affiche des images du set importé pour vérifier que tout a bien marché :

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


# pour montrer une image
def imshow(img):
    img = img / 2 + 0.5    #de-normaliser
    npimg = img.numpy()    #convertir en array
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# on loop sur un batch
dataiter = iter(train_loader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))
print('          '.join('%s' % cloud_classes[labels[j]] for j in range(4)))

Et maintenant on défini le réseau :

In [ ]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.fc1 = nn.Linear(32 * 32, 200)
            self.fc2 = nn.Linear(200, 200)
            self.fc3 = nn.Linear(200, 4)

        def forward(self, x):
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return F.log_softmax(input=x)
model = Net()
print(model)

Et l'optimiseur, toujours en SGD mais avec un learning rate 10 fois plus grand. Avec NLLL comme critère, on a rajouté une couche de softmax en sortie pour obtenir des log-proba.

In [ ]:
import torch.optim as optim

criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

On entraine :

In [ ]:
import time
start_time = time.time()
print("Started training")

epochs = 50
print_interval = 50 #prints every p_i*4
tempo = []
acc = []

for epoch in range(epochs):  # nbr epochs
    for batch_idx, (data, target) in enumerate(train_loader): #nbr batch,in,out
        data, target = Variable(data), Variable(target)

        #On resize pour la sortie
        data = data.view(-1, 32*32)

        #init l'entrainement
        optimizer.zero_grad()
        net_out = model(data)
        loss = criterion(net_out, target)
        loss.backward()
        optimizer.step()

        #afficher la progression
        if batch_idx % print_interval == 0:
            #le print statement le plus illisible du monde
            print('Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch+1, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.data[0]))
    tempo.append(epoch)
    acc.append(loss.data[0])
    
print("Finished training in  %.3f seconds " % (time.time() - start_time))

Et maintenant on teste :

In [ ]:
test_loss = 0
correct = 0

for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    
    #rescale
    data = data.view(-1, 32 * 32)
    net_out = model(data)
    
    #somme des pertes du batch
    test_loss += criterion(net_out, target).data[0]
    pred = net_out.data.max(1)[1] #prediction
    correct += pred.eq(target.data).sum() #output du réseau

test_loss /= len(test_loader.dataset) #loss = loss/length set
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

On sauvegarde le modèle :

In [ ]:
torch.save(model.state_dict(), "MODEL_trainEASY_pytorchMCV2")

In [ ]:
#joli plot du loss en fonction de l'epoch
import matplotlib.pyplot as plt
plt.plot(tempo, acc)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss variation - Easy training')
plt.savefig('Loss_easytraining.png')
plt.show()

Maintenant pour le set moyen :

In [1]:
import torch
import torchvision
from torchvision import transforms, datasets

#Import
data_transform = transforms.Compose(
    [transforms.Grayscale(),
     transforms.Resize((32,32)),
    transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5,0.5), (0.5,0.5,0.5))])

train_set = datasets.ImageFolder(root='clouds_medium',
                                transform=data_transform)
train_loader = torch.utils.data.DataLoader(train_set,
                                             batch_size=4, shuffle=True,
                                             num_workers=1)

test_set = datasets.ImageFolder(root='clouds_medium_test',
                                transform=data_transform)
test_loader = torch.utils.data.DataLoader(test_set,
                                             batch_size=4,shuffle=False,
                                             num_workers=1)

cloud_classes = ('0', 'pi/4', 'pi/2', '3pi/4')

#Model
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.fc1 = nn.Linear(32 * 32, 200)
            self.fc2 = nn.Linear(200, 200)
            self.fc3 = nn.Linear(200, 4)

        def forward(self, x):
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return F.log_softmax(input=x)
model = Net()

#Optimizer
import torch.optim as optim
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

#Training
import time
start_time = time.time()
print("Started training")

epochs = 50
print_interval = 50
tempo = []
acc = []

for epoch in range(epochs):  
    for batch_idx, (data, target) in enumerate(train_loader): 
        data, target = Variable(data), Variable(target)

        data = data.view(-1, 32*32)

        optimizer.zero_grad()
        net_out = model(data)
        loss = criterion(net_out, target)
        loss.backward()
        optimizer.step()

        if batch_idx % print_interval == 0:
            print('Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch+1, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.data[0]))
    tempo.append(epoch)
    acc.append(loss.data[0])
    
print("Finished training in  %.3f seconds " % (time.time() - start_time))

#Testing
test_loss = 0
correct = 0

for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    
    data = data.view(-1, 32 * 32)
    net_out = model(data)
    
    test_loss += criterion(net_out, target).data[0]
    pred = net_out.data.max(1)[1]
    correct += pred.eq(target.data).sum() 

test_loss /= len(test_loader.dataset) 
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

#Saving
torch.save(model.state_dict(), "MODEL_trainMEDIUM_pytorchMCV2")

#Plotting
import matplotlib.pyplot as plt
plt.plot(tempo, acc)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss variation - Medium training')
plt.savefig('Loss_medtraining.png')
plt.show()

Started training


/home/hugo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1 [0/2000 (0%)]	Loss: 1.405973
Epoch: 1 [200/2000 (10%)]	Loss: 1.472624
Epoch: 1 [400/2000 (20%)]	Loss: 1.472327
Epoch: 1 [600/2000 (30%)]	Loss: 1.401734
Epoch: 1 [800/2000 (40%)]	Loss: 1.275055
Epoch: 1 [1000/2000 (50%)]	Loss: 1.386950
Epoch: 1 [1200/2000 (60%)]	Loss: 1.367775
Epoch: 1 [1400/2000 (70%)]	Loss: 1.396947
Epoch: 1 [1600/2000 (80%)]	Loss: 1.370371
Epoch: 1 [1800/2000 (90%)]	Loss: 1.386262
Epoch: 2 [0/2000 (0%)]	Loss: 1.378950
Epoch: 2 [200/2000 (10%)]	Loss: 1.495698
Epoch: 2 [400/2000 (20%)]	Loss: 1.406986
Epoch: 2 [600/2000 (30%)]	Loss: 1.334477
Epoch: 2 [800/2000 (40%)]	Loss: 1.332101
Epoch: 2 [1000/2000 (50%)]	Loss: 1.336308
Epoch: 2 [1200/2000 (60%)]	Loss: 1.444397
Epoch: 2 [1400/2000 (70%)]	Loss: 1.479613
Epoch: 2 [1600/2000 (80%)]	Loss: 1.387455
Epoch: 2 [1800/2000 (90%)]	Loss: 1.377701
Epoch: 3 [0/2000 (0%)]	Loss: 1.344436
Epoch: 3 [200/2000 (10%)]	Loss: 1.402278
Epoch: 3 [400/2000 (20%)]	Loss: 1.485408
Epoch: 3 [600/2000 (30%)]	Loss: 1.194599
Epoch: 3 [800/2

Epoch: 20 [1400/2000 (70%)]	Loss: 1.858165
Epoch: 20 [1600/2000 (80%)]	Loss: 0.004092
Epoch: 20 [1800/2000 (90%)]	Loss: 0.002889
Epoch: 21 [0/2000 (0%)]	Loss: 0.000326
Epoch: 21 [200/2000 (10%)]	Loss: 0.351592
Epoch: 21 [400/2000 (20%)]	Loss: 0.612712
Epoch: 21 [600/2000 (30%)]	Loss: 0.000929
Epoch: 21 [800/2000 (40%)]	Loss: 0.002278
Epoch: 21 [1000/2000 (50%)]	Loss: 0.430526
Epoch: 21 [1200/2000 (60%)]	Loss: 0.004029
Epoch: 21 [1400/2000 (70%)]	Loss: 0.010619
Epoch: 21 [1600/2000 (80%)]	Loss: 0.100930
Epoch: 21 [1800/2000 (90%)]	Loss: 0.194808
Epoch: 22 [0/2000 (0%)]	Loss: 0.000309
Epoch: 22 [200/2000 (10%)]	Loss: 0.000480
Epoch: 22 [400/2000 (20%)]	Loss: 1.565876
Epoch: 22 [600/2000 (30%)]	Loss: 0.054466
Epoch: 22 [800/2000 (40%)]	Loss: 0.003180
Epoch: 22 [1000/2000 (50%)]	Loss: 0.523901
Epoch: 22 [1200/2000 (60%)]	Loss: 0.003319
Epoch: 22 [1400/2000 (70%)]	Loss: 0.169608
Epoch: 22 [1600/2000 (80%)]	Loss: 0.524826
Epoch: 22 [1800/2000 (90%)]	Loss: 0.027723
Epoch: 23 [0/2000 (0%)]	Los

Epoch: 40 [400/2000 (20%)]	Loss: 0.000223
Epoch: 40 [600/2000 (30%)]	Loss: 0.000166
Epoch: 40 [800/2000 (40%)]	Loss: 0.000204
Epoch: 40 [1000/2000 (50%)]	Loss: 0.000178
Epoch: 40 [1200/2000 (60%)]	Loss: 0.000121
Epoch: 40 [1400/2000 (70%)]	Loss: 0.000093
Epoch: 40 [1600/2000 (80%)]	Loss: 0.000629
Epoch: 40 [1800/2000 (90%)]	Loss: 0.000108
Epoch: 41 [0/2000 (0%)]	Loss: 0.000061
Epoch: 41 [200/2000 (10%)]	Loss: 0.005745
Epoch: 41 [400/2000 (20%)]	Loss: 0.000684
Epoch: 41 [600/2000 (30%)]	Loss: 0.000243
Epoch: 41 [800/2000 (40%)]	Loss: 0.000094
Epoch: 41 [1000/2000 (50%)]	Loss: 0.001021
Epoch: 41 [1200/2000 (60%)]	Loss: 0.000026
Epoch: 41 [1400/2000 (70%)]	Loss: 0.000490
Epoch: 41 [1600/2000 (80%)]	Loss: 0.000190
Epoch: 41 [1800/2000 (90%)]	Loss: 0.000250
Epoch: 42 [0/2000 (0%)]	Loss: 0.000287
Epoch: 42 [200/2000 (10%)]	Loss: 0.000052
Epoch: 42 [400/2000 (20%)]	Loss: 0.000413
Epoch: 42 [600/2000 (30%)]	Loss: 0.000304
Epoch: 42 [800/2000 (40%)]	Loss: 0.000426
Epoch: 42 [1000/2000 (50%)]	Lo

Et pour terminer avec le dur :

In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets

#Import
data_transform = transforms.Compose(
    [transforms.Grayscale(),
     transforms.Resize((32,32)),
    transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5,0.5), (0.5,0.5,0.5))])

train_set = datasets.ImageFolder(root='clouds_hard',
                                transform=data_transform)
train_loader = torch.utils.data.DataLoader(train_set,
                                             batch_size=4, shuffle=True,
                                             num_workers=1)

test_set = datasets.ImageFolder(root='clouds_hard_test',
                                transform=data_transform)
test_loader = torch.utils.data.DataLoader(test_set,
                                             batch_size=4,shuffle=False,
                                             num_workers=1)

cloud_classes = ('0', 'pi/4', 'pi/2', '3pi/4')

#Model
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.fc1 = nn.Linear(32 * 32, 200)
            self.fc2 = nn.Linear(200, 200)
            self.fc3 = nn.Linear(200, 4)

        def forward(self, x):
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return F.log_softmax(input=x)
model = Net()

#Optimizer
import torch.optim as optim
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

#Training
import time
start_time = time.time()
print("Started training")

epochs = 50
print_interval = 50
tempo = []
acc = []

for epoch in range(epochs):  
    for batch_idx, (data, target) in enumerate(train_loader): 
        data, target = Variable(data), Variable(target)

        data = data.view(-1, 32*32)

        optimizer.zero_grad()
        net_out = model(data)
        loss = criterion(net_out, target)
        loss.backward()
        optimizer.step()

        if batch_idx % print_interval == 0:
            print('Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch+1, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.data[0]))
    tempo.append(epoch)
    acc.append(loss.data[0])
    
print("Finished training in  %.3f seconds " % (time.time() - start_time))

#Testing
test_loss = 0
correct = 0

for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    
    data = data.view(-1, 32 * 32)
    net_out = model(data)
    
    test_loss += criterion(net_out, target).data[0]
    pred = net_out.data.max(1)[1]
    correct += pred.eq(target.data).sum() 

test_loss /= len(test_loader.dataset) 
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

#Saving
torch.save(model.state_dict(), "MODEL_trainHARD_pytorchMCV2")

#Plotting
import matplotlib.pyplot as plt
plt.plot(tempo, acc)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss variation - Hard training')
plt.savefig('Loss_hardtraining.png')
plt.show()